### Stock to Sales Ratio

In [0]:
%sql
SELECT 
    SUM(oi.total_price) AS stock_to_sales_ratio
FROM cpg_industry.silver.cpg_inventory inv
JOIN cpg_industry.silver.cpg_product p ON inv.product_id = p.product_id
JOIN cpg_industry.silver.cpg_order_items oi ON p.product_id = oi.product_id;


In [0]:
%sql
WITH avg_inventory AS (
    SELECT AVG(quantity_on_hand * p.unit_price) AS avg_inventory_value
    FROM cpg_industry.silver.cpg_inventory inv
    JOIN cpg_industry.silver.cpg_product p ON inv.product_id = p.product_id
)
SELECT 
    SUM(oi.quantity * p.unit_price) / NULLIF(avg_inventory_value, 0) AS inventory_turnover_rate
FROM cpg_industry.silver.cpg_order_items oi
JOIN cpg_industry.silver.cpg_product p ON oi.product_id = p.product_id
CROSS JOIN avg_inventory;


### Backorder Rate (%)

In [0]:
%sql
SELECT 
    ROUND(
        (SUM(CASE WHEN co.order_status = 'Returned' THEN coi.quantity ELSE 0 END) 
        / NULLIF(SUM(coi.quantity), 0)) * 100, 2
    ) AS backorder_rate_pct
FROM cpg_industry.silver.cpg_order_items coi
JOIN cpg_industry.silver.cpg_consumer_order co ON coi.order_id = co.order_id
WHERE co.order_date BETWEEN '2024-10-01' AND '2024-12-30';

### Days Sales in Inventory (DSI)


In [0]:
%sql
WITH avg_inventory AS (
    SELECT AVG(inv.quantity_on_hand * p.unit_price) AS avg_inventory_value
    FROM  cpg_industry.silver.cpg_inventory inv
    JOIN cpg_industry.silver.cpg_product p ON inv.product_id = p.product_id
),
cogs AS (
    SELECT SUM(cpi.total_price) AS total_cogs
    FROM cpg_industry.silver.cpg_order_items cpi
)
SELECT 
    ROUND(
        (avg_inventory.avg_inventory_value / NULLIF(cogs.total_cogs, 0)) * 365, 
        2
    ) AS days_sales_in_inventory
FROM avg_inventory, cogs;
 

### Perfect Order Rate (%)


In [0]:
%sql
SELECT 
    ROUND(
        (SUM(CASE WHEN order_status = 'Delivered' THEN 1 ELSE 0 END) 
        / NULLIF(COUNT(*), 0)) * 100, 
        2
    ) AS perfect_order_rate_pct
FROM cpg_industry.silver.cpg_consumer_order
WHERE order_date BETWEEN '2024-11-01' AND '2024-11-30';

In [0]:
%sql
show tables in cpg_industry.silver;

## Sales trend

In [0]:
%sql
SELECT 
    co.order_date,
    round(SUM(oi.total_price), 2) AS daily_sales_value
FROM cpg_industry.silver.cpg_order_items oi
JOIN cpg_industry.silver.cpg_consumer_order co 
    ON oi.order_id = co.order_id
WHERE co.order_status = 'Delivered'
GROUP BY co.order_date
ORDER BY co.order_date;
